In [2]:
# Import necessary Libraries
import numpy as np
import pandas as pd
import sys  
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 10, 8
from pprint import pprint
import seaborn as sns

In [3]:
# Load Data
foreign_mat = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Foreign_Matter.csv')
micro = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Micro_Screen.csv')
moisture = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Moisture_Content.csv')
solvent = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Solvent.csv')
potency = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/potencytests.csv') 
samples = pd.read_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Sample_lab_time.csv') 

In [4]:
# Create Test Type Column
foreign_mat['Test'] = 'Foreign Matter'
micro['Test'] = 'Micro Screen'
moisture['Test'] = 'Moisture Content'
solvent['Test'] = 'Solvent Screen'

# Combine Foreign Matter, Micro, Moisture Content and Solvent Screening
tests = pd.concat([foreign_mat, micro, moisture, solvent])
tests.sort_values('sample_id', ascending=True, inplace=True )
tests.head(20)

,sample_id,name,value,failure,location,orgid,Test
374,19,yeast_and_mold,8800.0,0,139,170,Micro Screen
373,19,bile_tolerant,0.0,0,139,170,Micro Screen
372,19,aerobic_bacteria,0.0,0,139,170,Micro Screen
74,19,moisture,6.0,0,139,170,Moisture Content
148,19,Stems,0.0,0,139,170,Foreign Matter
371,19,coliforms,0.0,0,139,170,Micro Screen
370,19,e_coli_and_salmonella,0.0,0,139,170,Micro Screen
149,19,Other,0.0,0,139,170,Foreign Matter
377,20,aerobic_bacteria,200.0,0,139,170,Micro Screen
379,20,coliforms,0.0,0,139,170,Micro Screen


In [5]:
# Subset Potency data in "...samples.csv" to lab and location level only
labs = pd.DataFrame(samples, columns=['sample_id', 'test_date', 'lab_license'])
labs.sort_values('sample_id', ascending=True, inplace=True )
labs.head()

,sample_id,test_date,lab_license
0,18,6/19/2014,4
1,19,6/19/2014,4
2,20,6/19/2014,4
3,21,6/19/2014,4
4,22,6/19/2014,4


In [6]:
# Merge lab data with lab locations
merge = pd.merge(tests, labs, on='sample_id', how='left')
merge.head()

,sample_id,name,value,failure,location,orgid,Test,test_date,lab_license
0,19,yeast_and_mold,8800.0,0,139,170,Micro Screen,6/19/2014,4
1,19,bile_tolerant,0.0,0,139,170,Micro Screen,6/19/2014,4
2,19,aerobic_bacteria,0.0,0,139,170,Micro Screen,6/19/2014,4
3,19,moisture,6.0,0,139,170,Moisture Content,6/19/2014,4
4,19,Stems,0.0,0,139,170,Foreign Matter,6/19/2014,4


In [1]:
# Ignore this for now
# merge.to_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/lab_producer.csv', index=False)

In [7]:
# Count number of fails by lab and test type
fails = merge.groupby(['lab_license', 'Test'])['failure'].sum().reset_index()
#Lab_group = spanish.sort_values('spanish', ascending=False)
fails.head()

,lab_license,Test,failure
0,1,Foreign Matter,0
1,1,Micro Screen,148
2,1,Moisture Content,4
3,1,Solvent Screen,0
4,2,Foreign Matter,33


In [8]:
# Count the total number of tests at each lab by type
test_tot = merge.groupby(['lab_license', 'Test'])['failure'].count().reset_index()
test_tot.head()

,lab_license,Test,failure
0,1,Foreign Matter,764
1,1,Micro Screen,3810
2,1,Moisture Content,378
3,1,Solvent Screen,25
4,2,Foreign Matter,1460


In [9]:
# Merge number of fails with total number of tests
fail_merge = pd.merge(fails, test_tot, on=['lab_license','Test'], how='left')
fail_merge = fail_merge.rename(columns = {
    'failure_x':'Num Fail', 'failure_y':'Total Tests'})
# Calculate the fail rate for each lab and test type
fail_merge['Fail Rate'] = fail_merge['Num Fail']/fail_merge['Total Tests']
fail_merge.tail()

,lab_license,Test,Num Fail,Total Tests,Fail Rate
67,17,Solvent Screen,7,28,0.25000
68,18,Foreign Matter,0,196,0.00000
69,18,Micro Screen,37,630,0.05873
70,18,Moisture Content,0,98,0.00000
71,18,Solvent Screen,0,12,0.00000


In [10]:
# Calculate the fail rate for each lab and test type
fail_merge['Fail Rate'] = fail_merge['Num Fail']/fail_merge['Total Tests']
fail_merge.tail()

,lab_license,Test,Num Fail,Total Tests,Fail Rate
67,17,Solvent Screen,7,28,0.25000
68,18,Foreign Matter,0,196,0.00000
69,18,Micro Screen,37,630,0.05873
70,18,Moisture Content,0,98,0.00000
71,18,Solvent Screen,0,12,0.00000


In [11]:
# Export to CSV
# From here I did all of the analyses for Beau's presentation in R. Yes, this is annoying. No one else was using python so I 
# switched to R to make it easier for everyone.
fail_merge.to_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/fail_merge.csv', index=True)

In [10]:
# The code from here on is for other analyses that did not make it into the presentation for Beau
# Create a pivot table
Labs_pivot2 = merge.pivot_table(values=['failure'], index=['lab_license'], columns=['Test'], aggfunc=[np.sum,len])
Labs_pivot2

sum                                               \
                   failure                                                
Test        Foreign Matter Micro Screen Moisture Content Solvent Screen   
lab_license                                                               
1                      0.0        148.0              4.0            0.0   
2                     33.0        344.0              9.0            3.0   
3                      8.0       3448.0             34.0          175.0   
4                      0.0       4537.0            186.0          244.0   
5                      0.0         55.0              0.0            6.0   
6                      1.0        476.0             83.0            1.0   
7                      0.0        103.0             14.0            0.0   
8                      2.0        290.0              1.0            0.0   
9                      0.0       1344.0              6.0           67.0   
10                     0.0        718.0             28.0           11.0   
11                     0.0        149.0             26.0            3.0   
12                    73.0       1311.0             27.0           42.0   
13                     0.0        299.0             10.0            0.0   
14                     3.0        589.0              9.0            0.0   
15                     0.0        235.0              1.0            2.0   
16                     1.0        133.0              1.0            0.0   
17                     0.0        174.0              2.0            7.0   
18                     0.0         37.0              0.0            0.0   
123456                 0.0         10.0              0.0            NaN   

                       len                                               
                   failure                                               
Test        Foreign Matter Micro Screen Moisture Content Solvent Screen  
lab_license                                                              
1                    764.0       3810.0            378.0           25.0  
2                   1460.0       9877.0            724.0         1250.0  
3                  66484.0     189285.0          32948.0         3142.0  
4                  18298.0      62251.0           9083.0         2538.0  
5                   1712.0       4815.0            851.0           92.0  
6                   5246.0      17032.0           2620.0           64.0  
7                  24342.0      68570.0          12071.0          862.0  
8                   5938.0      18365.0           2949.0          113.0  
9                  20944.0      72140.0          10443.0         2871.0  
10                  5286.0      14605.0           2628.0          298.0  
11                  1554.0       4035.0            761.0            4.0  
12                 27840.0      76130.0          13863.0          887.0  
13                  1312.0       3500.0            656.0            2.0  
14                  2278.0       7880.0           1138.0          351.0  
15                 21052.0      57620.0          10526.0          886.0  
16                   820.0       2280.0            410.0           25.0  
17                   712.0       1940.0            356.0           28.0  
18                   196.0        630.0             98.0           12.0  
123456                 8.0         20.0              4.0            NaN

In [38]:
Labs_pivot2.to_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/Lab_fail_rate.csv', index=True)

In [11]:
# df[df['languages'] != 'x'].astype(str)
solvent = merge[(merge["Test"] == "Solvent Screen")]
solvent.head()

,sample_id,name,value,failure,location,orgid,Test,test_date,lab_license
10070,1370,residual_solvent,50.0,0,168,214,Solvent Screen,9/13/2014,3
10078,1371,residual_solvent,50.0,0,168,214,Solvent Screen,9/13/2014,3
12163,1709,residual_solvent,50.0,0,168,214,Solvent Screen,9/20/2014,3
12173,1710,residual_solvent,50.0,0,168,214,Solvent Screen,9/20/2014,3
14652,2111,residual_solvent,60.0,0,433,556,Solvent Screen,9/30/2014,4


In [12]:
# Crosstab to show the number of Solvent tests each producer did at each lab
solvent_cross = pd.crosstab(solvent["location"],solvent["lab_license"],margins=True)
solvent_cross

lab_license,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,All
location,,,,,,,,,,,,,,,,,,,
126,0,0,74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,74
139,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
148,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
153,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
161,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
167,0,55,147,69,0,0,0,0,0,0,0,0,0,0,0,0,0,0,271
168,0,0,95,2,0,0,0,0,24,0,0,0,0,0,0,0,0,0,121
169,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
180,0,0,21,3,0,0,12,0,0,0,0,0,0,0,16,0,0,0,52


In [42]:
solvent_cross.to_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/solvent_cross.csv', index=True)

In [14]:
three_tests = merge[(merge["Test"] == "Foreign Matter") | (merge["Test"] == "Micro Screen") | (merge["Test"] == "Moisture Content")]
three_tests.head(20)

,sample_id,name,value,failure,location,orgid,Test,test_date,lab_license
0,19,yeast_and_mold,8800.0,0,139,170,Micro Screen,6/19/2014,4
1,19,bile_tolerant,0.0,0,139,170,Micro Screen,6/19/2014,4
2,19,aerobic_bacteria,0.0,0,139,170,Micro Screen,6/19/2014,4
3,19,moisture,6.0,0,139,170,Moisture Content,6/19/2014,4
4,19,Stems,0.0,0,139,170,Foreign Matter,6/19/2014,4
5,19,coliforms,0.0,0,139,170,Micro Screen,6/19/2014,4
6,19,e_coli_and_salmonella,0.0,0,139,170,Micro Screen,6/19/2014,4
7,19,Other,0.0,0,139,170,Foreign Matter,6/19/2014,4
8,20,aerobic_bacteria,200.0,0,139,170,Micro Screen,6/19/2014,4
9,20,coliforms,0.0,0,139,170,Micro Screen,6/19/2014,4


In [18]:
three_test_cross = pd.crosstab(three_tests["location"],three_tests["lab_license"],margins=True)
three_test_cross

lab_license,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,123456,All
location,,,,,,,,,,,,,,,,,,,,
124,0,0,0,192,0,0,0,0,0,0,0,853,0,10,0,0,0,0,0,1055
126,0,32,2224,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2256
127,0,16,570,184,0,0,584,0,656,0,0,0,0,0,0,0,0,0,0,2010
139,0,0,1040,280,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1320
140,0,0,0,379,0,0,417,517,48,0,0,0,0,0,0,0,0,0,0,1361
141,16,0,1720,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1736
142,0,0,236,0,0,0,328,0,0,0,0,0,0,0,0,0,0,0,0,564
143,0,0,400,0,0,0,0,0,0,0,0,2677,0,0,0,0,0,0,0,3077
144,32,0,1144,0,0,0,770,0,0,0,0,0,0,0,0,0,0,0,0,1946


In [19]:
three_test_cross.to_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/three_test_cross.csv', index=True)

In [21]:
fm_cross_merge = merge[(merge["Test"] == "Foreign Matter")]
fm_cross_merge.head()

,sample_id,name,value,failure,location,orgid,Test,test_date,lab_license
4,19,Stems,0.0,0,139,170,Foreign Matter,6/19/2014,4
7,19,Other,0.0,0,139,170,Foreign Matter,6/19/2014,4
13,20,Other,0.0,0,139,170,Foreign Matter,6/19/2014,4
15,20,Stems,0.0,0,139,170,Foreign Matter,6/19/2014,4
18,21,Stems,0.0,0,139,170,Foreign Matter,6/19/2014,4


In [22]:
fm_cross = pd.crosstab(fm_cross_merge["location"],fm_cross_merge["lab_license"],margins=True)
fm_cross

lab_license,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,123456,All
location,,,,,,,,,,,,,,,,,,,,
124,0,0,0,48,0,0,0,0,0,0,0,212,0,0,0,0,0,0,0,260
126,0,8,466,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,474
127,0,4,144,46,0,0,146,0,164,0,0,0,0,0,0,0,0,0,0,504
139,0,0,260,70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,330
140,0,0,0,66,0,0,92,128,12,0,0,0,0,0,0,0,0,0,0,298
141,4,0,432,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,436
142,0,0,62,0,0,0,82,0,0,0,0,0,0,0,0,0,0,0,0,144
143,0,0,100,0,0,0,0,0,0,0,0,648,0,0,0,0,0,0,0,748
144,8,0,286,0,0,0,190,0,0,0,0,0,0,0,0,0,0,0,0,484


In [23]:
fm_cross.to_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/fm_cross.csv', index=True)

In [24]:
micro_cross_merge = merge[(merge["Test"] == "Micro Screen")]
micro_cross_merge.head(20)

,sample_id,name,value,failure,location,orgid,Test,test_date,lab_license
0,19,yeast_and_mold,8800.0,0,139,170,Micro Screen,6/19/2014,4
1,19,bile_tolerant,0.0,0,139,170,Micro Screen,6/19/2014,4
2,19,aerobic_bacteria,0.0,0,139,170,Micro Screen,6/19/2014,4
5,19,coliforms,0.0,0,139,170,Micro Screen,6/19/2014,4
6,19,e_coli_and_salmonella,0.0,0,139,170,Micro Screen,6/19/2014,4
8,20,aerobic_bacteria,200.0,0,139,170,Micro Screen,6/19/2014,4
9,20,coliforms,0.0,0,139,170,Micro Screen,6/19/2014,4
10,20,bile_tolerant,0.0,0,139,170,Micro Screen,6/19/2014,4
11,20,yeast_and_mold,7100.0,0,139,170,Micro Screen,6/19/2014,4
14,20,e_coli_and_salmonella,0.0,0,139,170,Micro Screen,6/19/2014,4


In [25]:
micro_cross = pd.crosstab(micro_cross_merge["location"],micro_cross_merge["lab_license"],margins=True)
micro_cross

lab_license,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,123456,All
location,,,,,,,,,,,,,,,,,,,,
124,0,0,0,120,0,0,0,0,0,0,0,535,0,10,0,0,0,0,0,665
126,0,20,1525,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1545
127,0,10,360,115,0,0,365,0,410,0,0,0,0,0,0,0,0,0,0,1260
139,0,0,650,175,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,825
140,0,0,0,280,0,0,280,325,30,0,0,0,0,0,0,0,0,0,0,915
141,10,0,1080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1090
142,0,0,145,0,0,0,205,0,0,0,0,0,0,0,0,0,0,0,0,350
143,0,0,250,0,0,0,0,0,0,0,0,1705,0,0,0,0,0,0,0,1955
144,20,0,715,0,0,0,485,0,0,0,0,0,0,0,0,0,0,0,0,1220


In [26]:
micro_cross.to_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/micro_cross.csv', index=True)

In [27]:
mc_cross_merge = merge[(merge["Test"] == "Moisture Content")]
mc_cross_merge.head()

,sample_id,name,value,failure,location,orgid,Test,test_date,lab_license
3,19,moisture,6.0,0,139,170,Moisture Content,6/19/2014,4
12,20,moisture,7.0,0,139,170,Moisture Content,6/19/2014,4
22,21,moisture,9.0,0,139,170,Moisture Content,6/19/2014,4
31,22,moisture,6.0,0,139,170,Moisture Content,6/19/2014,4
39,23,moisture,9.0,0,139,170,Moisture Content,6/20/2014,4


In [28]:
mc_cross = pd.crosstab(mc_cross_merge["location"],mc_cross_merge["lab_license"],margins=True)
mc_cross

lab_license,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,123456,All
location,,,,,,,,,,,,,,,,,,,,
124,0,0,0,24,0,0,0,0,0,0,0,106,0,0,0,0,0,0,0,130
126,0,4,233,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,237
127,0,2,66,23,0,0,73,0,82,0,0,0,0,0,0,0,0,0,0,246
139,0,0,130,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,165
140,0,0,0,33,0,0,45,64,6,0,0,0,0,0,0,0,0,0,0,148
141,2,0,208,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,210
142,0,0,29,0,0,0,41,0,0,0,0,0,0,0,0,0,0,0,0,70
143,0,0,50,0,0,0,0,0,0,0,0,324,0,0,0,0,0,0,0,374
144,4,0,143,0,0,0,95,0,0,0,0,0,0,0,0,0,0,0,0,242


In [29]:
mc_cross.to_csv('C:/Users/Krista/Documents/CMU/Spring 2017/Systems/Data/mc_cross.csv', index=True)